<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Rozpoznawanie-twarzy" data-toc-modified-id="Rozpoznawanie-twarzy-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Rozpoznawanie twarzy</a></span><ul class="toc-item"><li><span><a href="#Dane---olivetti-faces" data-toc-modified-id="Dane---olivetti-faces-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Dane - olivetti faces</a></span></li><li><span><a href="#przygotowanie-danych-treningowych-oraz-testowych" data-toc-modified-id="przygotowanie-danych-treningowych-oraz-testowych-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>przygotowanie danych treningowych oraz testowych</a></span></li><li><span><a href="#Tworzymy-model-sieci-konwolucyjnej" data-toc-modified-id="Tworzymy-model-sieci-konwolucyjnej-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Tworzymy model sieci konwolucyjnej</a></span></li><li><span><a href="#callbacks" data-toc-modified-id="callbacks-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>callbacks</a></span></li><li><span><a href="#trening" data-toc-modified-id="trening-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>trening</a></span></li><li><span><a href="#sprawdzamy-tensorboard" data-toc-modified-id="sprawdzamy-tensorboard-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>sprawdzamy tensorboard</a></span></li></ul></li><li><span><a href="#Aktywacja-funkcją-&quot;sigmoid&quot;" data-toc-modified-id="Aktywacja-funkcją-&quot;sigmoid&quot;-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Aktywacja funkcją "sigmoid"</a></span></li><li><span><a href="#Dodatkowe-źródła-informacji" data-toc-modified-id="Dodatkowe-źródła-informacji-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Dodatkowe źródła informacji</a></span></li></ul></div>

In [ ]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tensorflow as tf
from tensorflow.keras import layers
import datetime
from sklearn.metrics import f1_score

# jeśli nie korzystasz z poniższej paczki, to wykomentuj te dwie linie poniżej
from jupyterthemes import jtplot
jtplot.style(theme="monokai", context="notebook", ticks=True, grid=False)

# Powtórka
- implementacja sieci konwolucyjnej w KERASIE
- badanie i zapobieganie przeuczeniu
- wzbogacenie modelu o _batch normalization_
- wzbogacenie procesu uczenia o regularyzację
- sprawdzenie różnicy w uczeniu podczas zastosowania aktywacji 'sigmoid'

## Rozpoznawanie twarzy

### Dane - olivetti faces

In [ ]:
from sklearn.datasets import fetch_olivetti_faces
faces = fetch_olivetti_faces()

In [ ]:
print(faces.DESCR)

In [ ]:
plt.imshow(faces.images[0])
plt.show()

In [ ]:
faces.images.shape

### przygotowanie danych treningowych oraz testowych

In [ ]:
x = faces.images.reshape(-1,64,64,1)

In [ ]:
y = faces.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
# sprawdzamy jakie są i ile jest klas
print("Jakie klasy: ", np.unique(y_train))
classNum = np.unique(y_train).shape[0]
print("ile jest klas: ", classNum)

### Tworzymy model sieci konwolucyjnej

In [ ]:
def createSeqModel(activation="relu", learning_rate=0.001):
    """
    funkcja do tworzenia modelu
    :param activation: rodzaj funkcji aktywacji na wszystkich warstwach
    :param learning_rate: współczynnik 'prędkości' uczenia
    """
    # pusty model
    model = tf.keras.models.Sequential()

    # pierwsza warstwa jako samo wejście
    model.add(tf.keras.layers.InputLayer(input_shape=(64,64,1)))
    # pierwsza ukryta, konwolucyjna
    model.add(tf.keras.layers.Conv2D(filters=16,
                                     kernel_size=[3,3],
                                     padding="same",
                                     activity_regularizer=tf.keras.regularizers.l1_l2(l1=0.001,
                                                                                     l2=0.001)
                                    ))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(activation))
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.Dropout(0.2))
    # druga ukryta, konwolucyjna
    model.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=[3,3],
                                     padding="same",
                                     activity_regularizer=tf.keras.regularizers.l1_l2(l1=0.001,
                                                                                     l2=0.001)
                                    ))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(activation))
    model.add(tf.keras.layers.MaxPool2D())
    model.add(tf.keras.layers.Dropout(0.2))
    # spłaszczamy pod warstwy fully connected(Dense)
    model.add(tf.keras.layers.Flatten())
    # czwarta ukryta fully connetced
    model.add(tf.keras.layers.Dense(64,
                                    activity_regularizer=tf.keras.regularizers.l1_l2(l1=0.001,
                                                                                     l2=0.001)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation(activation))
    model.add(tf.keras.layers.Dropout(0.5))
    # wyjściowa, zadaniem jest klasyfikacja
    model.add(tf.keras.layers.Dense(classNum,
                                    activity_regularizer=tf.keras.regularizers.l1_l2(l1=0.001,
                                                                                     l2=0.001)))
    # aktywacja jako softmax, aby uzyskać rozkład prawdopodbieństwa
    model.add(tf.keras.layers.Softmax())
    # żadnego dropout'u, nie chcemy abym nam wyzerowało wartość
    
    # drukujemy opis modelu
    print(model.summary())
    
    # wybieramy optimalizator, polecam pobawić się z różnymi typami oraz różnymi wartościami learning rate'u
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    # kompilujemy
    model.compile(optimizer=opt, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              metrics=["accuracy"])
    
    return model

### callbacks

In [ ]:
def createCallbacks(model, modelName):
    """
    funkcja tworząca odpowiednie zestawienie callbacków
    niektóre do podsumowania, a niektóre do kontroli treningu
    :param model: model sieci w kerasie
    :param modelName: nazwa modelu, w celu rozróżnienia w tensorboard
    """
    # dodajemy zapisywanie logów do tensorboard'a
    # katalog nazywa się jak model wraz z czasem utworzenia
    log_dir = "logs\\" + modelName + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    
    ###############
    # główny callback odnoszący się do samego modelu, wag i nmetryk
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
    ###############
    # callback kontrolujący spadek błędu na datasecie validacyjnym
    # i zatrzymujący uczenie jeśli się nie poprawi przez ileśtam epok
    es_callback = tf.keras.callbacks.EarlyStopping(patience=20)
    
    ###############
    # callback do gradientów
    # utworzenie zapisywacza wartości
    file_writer_grads = tf.summary.create_file_writer(log_dir+"\\grads")
    # funkcja licząca gradienty
    def calcGrads(epoch, logs):
        with tf.GradientTape() as tape:
            loss = model(x_train[:500])
        grads = tape.gradient(loss, model.trainable_weights)
        
        # Log gradients
        with file_writer_grads.as_default():
            #g_sum = tf.reduce_sum(grads[-1])
            #tf.summary.histogram("grads", grads[-1], step=epoch)
            for g, w in zip(grads, model.trainable_weights):
                tf.summary.histogram(w.name, g, step=epoch)
    
    # utworzenie callback'a od zapisywania gradientów
    # na koniec każdej epoki
    grads_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=calcGrads)
    
    ###############
    # callback od tworzenia logów z f1_score
    # utworzenie zapisywacza wartości
    file_writer_f1 = tf.summary.create_file_writer(log_dir+"\\f1")

    # funkcja licząca
    def log_f1_score(epoch, logs):
        y_pred = model.predict(x_test)
        y_pred = np.argmax(y_pred, axis=1)

        f1Val = f1_score(y_test, y_pred, average='micro')

        # Log the confusion matrix as an image summary.
        with file_writer_f1.as_default():
            tf.summary.scalar("f1 score", f1Val, step=epoch)

    # utworzenie callback'a od zapisywania f1_score'a
    # na koniec każdej epoki
    f1_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_f1_score)
    
    # zwracamy listę callbacków
    return [tensorboard_callback, es_callback, grads_callback, f1_callback]

### trening

In [ ]:
def train(model, batchSize, epochsNum, callbacks):
    """
    funkcja wykonująca trening
    :param model: model do wykonania treningu
    :param batchSize: wielkość pojedynczej paczki danych podczas treningu
    :param epochsNum: ilość epok treningu, czyli ile razy po całym datasecie przelecimy
    :param callbacks: lista callbacków zapisywujących podsumowanie treningu
    """
    history = model.fit(x_train, y_train, 
                        batch_size=batchSize, epochs=epochsNum, verbose=1, 
                        initial_epoch=0,
                        validation_data=(x_test, y_test), 
                        callbacks=callbacks)
    
    # rysujemy wykres błędu
    plt.plot(history.history["loss"])
    plt.title("loss")
    plt.show()

In [ ]:
# tworzymy model sieci, póki co lecimy na difolcie
model = createSeqModel()

In [ ]:
# tworzymy podsumowania
callbacks = createCallbacks(model, "reluRegularizedLowCNN")

In [ ]:
# trenujemy model
train(model, 20, 200, callbacks)

### sprawdzamy tensorboard

In [ ]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard

In [ ]:
# uruchomienie serwera, jeżeli z poziomu notebooka nie działa, to należy uruchomić z cmd, ale bez '%'
#%tensorboard --logdir logs

## Aktywacja funkcją "sigmoid"
- poprzednie wyniki porównujemy z siecią opartą o aktywację "sigmoid"

In [ ]:
# tworzymy model sieci z "sigmoid"
modelSigmoid = createSeqModel("sigmoid")

In [ ]:
# tworzymy podsumowania
callbacks = createCallbacks(model, "sigmoidCNN")

In [ ]:
# trenujemy model
train(modelSigmoid, 50, 200, callbacks)

## Dodatkowe źródła informacji
- [Wyjaśnienie problemu zanikającego gradientu](https://towardsdatascience.com/the-problem-of-vanishing-gradients-68cea05e2625)
- [Zastosowanie regularyzacji w praktyce](https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/)
- [Ciekawe zestawienie w działaniu warstwy BatchNorm oraz Regularization.](https://blog.janestreet.com/l2-regularization-and-batch-norm/)
- [Dobre wyjaśnienie na czym polega oszukiwanie sieci](https://medium.com/@ageitgey/machine-learning-is-fun-part-8-how-to-intentionally-trick-neural-networks-b55da32b7196)